In [46]:
from bs4 import BeautifulSoup
import requests
from datetime import datetime

headers = { 'User-Agent': 'Mozilla/5.0 (Windows NT 6.0; WOW64; rv:24.0) Gecko/20100101 Firefox/24.0' }

base_url = 'http://www.rohcollections.org.uk/SearchResults.aspx?searchtype=performance&yearfrom=1732&genre=Opera&page='

In [47]:
listings = []

for page in range(233):
    
    if page%50==0:
        print(page)
        
    url = base_url + '&i=' + str(page)
    r  = requests.get(url, headers=headers)
    soup = BeautifulSoup(r.text, "html5lib")
    results = soup.find(attrs={'class': 'results'}).find_all('tr')
    
    show = True
    for result in results[1:]:
        listing  = []
        items    = result.find_all('td')
        work     = items[0].get_text()
        date     = datetime.strptime(items[1].get_text(), '%d %B %Y').date()             
        
        #go to the listing and then to the root work in order to get composer
        work_url = 'http://www.rohcollections.org.uk/' + items[1].find('a').get('href')        
        work_r    = requests.get(work_url, headers=headers)
        work_soup = BeautifulSoup(work_r.text, "html5lib")
        work_url  = work_soup.find(attrs={'id': 'ucHeader_uiSubSubDirectoryLink'}).get('href')
        
        #page of the work itself
        work_url = 'http://www.rohcollections.org.uk' + work_url
        work_r    = requests.get(work_url, headers=headers)
        work_soup = BeautifulSoup(work_r.text, "html5lib")
        details = work_soup.find(attrs={'class': 'result'})
        table = list(zip(details.find_all('th'),details.find_all('td')))
        
        composer, librettist, language, based_on = '','','',''
        for row in table:
            if row[0].get_text() == 'Composer:':
                composer = row[1].get_text()
            if row[0].get_text() == 'Librettist:':
                librettist = row[1].get_text()            
            if row[0].get_text() == 'Language:':
                language = row[1].get_text()  
            if row[0].get_text() in ('Based on:', 'After:'):
                based_on = row[1].get_text()                  
                        
        listing.append(work)
        listing.append(date)
        listing.append(composer)
        listing.append(librettist)
        listing.append(language)
        listing.append(based_on)        
        listings.append(listing)
            
# #Saving results into dataframe and folder            
import pandas as pd                     
headers = ['work', 'date', 'composer', 'librettist', 'based_on', 'language', ]
df = pd.DataFrame(listings, columns=headers)
df.to_csv('../../data/processed/roh_listings.csv')            

0
50
100
150
200


In [39]:
url = 'http://www.rohcollections.org.uk/work.aspx?work=10402'
r  = requests.get(url, headers=headers)
soup = BeautifulSoup(r.text, "html5lib")
results = soup.find_all(attrs={'class': 'results'})

In [44]:
# results

In [16]:
row.find_all('td')

[<td>The Fairy Queen</td>,
 <td><a href="performance.aspx?performance=3696&amp;row=0&amp;searchtype=performance&amp;page=0&amp;yearfrom=1732&amp;genre=Opera">12 December 1946</a></td>,
 <td>Evening</td>,
 <td>Sadler's Wells Ballet, Covent Garden Opera Company</td>]